In [12]:
# Import necessary libraries
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
from ase.build import bulk, surface, make_supercell
from ase.io import write
from ase import Atoms
import numpy as np
from scipy.spatial.transform import Rotation as R
from copy import deepcopy
from ase.build import sort

In [62]:
# Constants
SURFACE_COVERAGE = 4.0  # Desired density of 4MePACz molecules per nm^2

# Initialize the Materials Project REST API client
with MPRester("M5gsyRYge3DgdJjf4NRxiDpXOvDHE3oB") as mpr:
    # Fetch the structure for NiO with mp-19009
    nio_structure = mpr.get_structure_by_material_id("mp-19009")

# Convert pymatgen structure to ASE Atoms
positions = nio_structure.cart_coords
numbers = [site.specie.number for site in nio_structure]
atoms = Atoms(numbers=numbers, positions=positions, cell=nio_structure.lattice.matrix, pbc=True)

# Build the NiO rock salt structure using ASE
nio_rocksalt = bulk('NiO', 'rocksalt', a=4.17)

# Create a (1 0 -1) slab of NiO
slab = surface(nio_rocksalt, indices=(1, 1, 0), layers=8, vacuum=15)

# Create a supercell from the slab
supercell = make_supercell(slab, [[9, 0, 0], [0, 9, 0], [0, 0, 1]])

# Write the ASE atoms object to an XYZ file
write("NiO_supercell.xyz", supercell, format="xyz")

# Convert XYZ to PDB using Open Babel
!obabel NiO_supercell.xyz -O NiO_supercell.pdb

# Load the LigParGen topology and XYZ files for 4-MePAcZ
ligpargen_itp = './INPUT/INPUT.itp'
ligpargen_xyz = './INPUT/INPUT.xyz'

# Load the topology and coordinates using ParmEd
p_ligand_structure = pmd.load_file(ligpargen_itp)
p_ligand_structure.coordinates = pmd.load_file(ligpargen_xyz).coordinates


# Identify the phosphorus atom in the ligand
phosphorus_atom = next(atom for atom in p_ligand_structure.atoms if atom.element == 15)

# Calculate the surface area in nm^2
surface_area_angstroms = slab.cell[0, 0] * slab.cell[1, 1]  # Å^2
surface_area_nm = surface_area_angstroms * 1e-1  # convert to nm^2

# Calculate the number of 4MePACz molecules to place on the surface
num_molecules = int(SURFACE_COVERAGE * surface_area_nm)


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

1 molecule converted


In [64]:
#The ordering of the script appears to be wrong, request GPT to fix it****!!!!

# Calculate the number of 4MePACz molecules to place on the surface
num_molecules = int(SURFACE_COVERAGE * surface_area_nm)

if num_molecules == 0:
    raise ValueError("The number of molecules to place on the surface is zero. Check the surface coverage and surface area.")

# Calculate grid dimensions
grid_size_x = int(np.ceil(np.sqrt(num_molecules * supercell.cell[0, 0] / supercell.cell[1, 1])))
grid_size_y = int(np.ceil(np.sqrt(num_molecules * supercell.cell[1, 1] / supercell.cell[0, 0])))

spacing_x = supercell.cell[0, 0] / grid_size_x
spacing_y = supercell.cell[1, 1] / grid_size_y

# Calculate the height above the surface for placement
height_above_surface = 2.0  # Adjust as needed

# Place molecules on the surface
molecule_count = 0
for ix in range(grid_size_x):
    for iy in range(grid_size_y):
        if molecule_count >= num_molecules:
            break
        
        # Copy the ligand structure
        ligand_copy = deepcopy(p_ligand_structure)
        
        # Calculate the position for the molecule in the grid
        x = ix * spacing_x + spacing_x / 2
        y = iy * spacing_y + spacing_y / 2
        z = max(supercell.positions[:, 2]) + height_above_surface  # Fixed height above the slab

        
        translation_vector = [x, y, 0]
        ligand_copy.coordinates += translation_vector
        
        # Calculate the vector from the phosphorus atom to the target point on the NiO surface
        target_position = np.array([x, y, max(supercell.positions[:, 2]) + 1.5])
        current_position = ligand_copy.coordinates[phosphorus_atom.idx]
        align_vector = target_position - current_position

        # Determine the current direction of the phosphorus atom
        current_vector = np.array([0, 0, 1])  # Assuming the molecule is initially oriented along the z-axis

        # Calculate the rotation matrix to align the current vector with the align vector
        rotation_axis = np.cross(current_vector, align_vector)
        rotation_angle = np.arccos(np.dot(current_vector, align_vector) / (np.linalg.norm(current_vector) * np.linalg.norm(align_vector)))
        rotation_matrix = R.from_rotvec(rotation_angle * rotation_axis / np.linalg.norm(rotation_axis)).as_matrix()

        # Apply the rotation to the entire 4MePACz molecule
        ligand_copy.coordinates = np.dot(ligand_copy.coordinates - current_position, rotation_matrix) + current_position

        # Add the ligand to the supercell
        for atom in ligand_copy.atoms:
            supercell += Atoms([atom.element], positions=[ligand_copy.coordinates[atom.idx]])
        
        molecule_count += 1

# Custom function to write the final structure to an XYZ file
def write_xyz(filename, structure):
    with open(filename, 'w') as f:
        f.write(f"{len(structure)}\n")
        f.write("XYZ file\n")
        for atom in structure:
            f.write(f"{atom.symbol} {atom.position[0]} {atom.position[1]} {atom.position[2]}\n")

#Sort the supercell
supercell=sort(supercell)

# Write the final structure using the custom function
write_xyz("./LAMMPS_Output/populated_surface.xyz", supercell)

#Write to POSCAR
#write("./LAMMPS_Output/populated_surface.xyz", supercell)
write("./LAMMPS_Output/POSCAR", supercell)